In [2]:
from pymavlink import mavutil
import time
from datetime import datetime, timezone
from influxdb import InfluxDBClient


In [3]:
mlog = mavutil.mavlink_connection(r"C:\Users\Alton\Downloads\2020-11-04 10-21-37.tlog")

In [4]:
client = InfluxDBClient(host='192.168.0.99', port=8086)
###client.drop_database('pyexample')
client.create_database('pyexample')
print(client.get_list_database())
client.switch_database('pyexample')

[{'name': '_internal'}, {'name': 'pyexample'}]


In [5]:
mlog.rewind()

while (True):
    msg = mlog.recv_msg().to_dict()
    mavpackettype = msg["mavpackettype"]

    #if not (msg["mavpackettype"] == "PARAM_VALUE"):

    if (mavpackettype == "AHRS3"):
    #if (mavpackettype == "VFR_HUD"):
    #if (mavpackettype == "WIND"):
    #if (mavpackettype == "BATTERY_STATUS"):
    #if (mavpackettype == "BATTERY2"):
    #if (mavpackettype == "SERVO_OUTPUT_RAW"):
    #if (mavpackettype == "NAMED_VALUE_FLOAT"): # got to make sure to not loop flatten keys

        msg.pop('mavpackettype', None)

        keys_need_flattened = []
        for key in msg:
            if (isinstance(msg[key], list)):
                keys_need_flattened.append(key)

        if len(keys_need_flattened)!=0:
            for key in keys_need_flattened:
                temp_list = msg[key]
                for idx, value in enumerate(temp_list):
                    new_key = key+"["+str(idx)+"]"
                    msg[new_key] = value
            msg.pop(key, None)

        time_utc = datetime.utcfromtimestamp(mlog.timestamp).isoformat()
        dict_body = {}
        dict_body["measurement"] = mavpackettype
        dict_body["time"] = time_utc
        dict_body["fields"] = msg
        json_body = [dict_body]

        # WRITE TO DB
        client.write_points(json_body)
        print(time_utc)



-04T20:52:44.122096
2020-11-04T20:52:44.411992
2020-11-04T20:52:44.925055
2020-11-04T20:52:45.214076
2020-11-04T20:52:45.474778
2020-11-04T20:52:45.871985
2020-11-04T20:52:46.148121
2020-11-04T20:52:46.485053
2020-11-04T20:52:46.837603
2020-11-04T20:52:47.191116
2020-11-04T20:52:47.676164
2020-11-04T20:52:47.843001
2020-11-04T20:52:48.149248
2020-11-04T20:52:48.454328
2020-11-04T20:52:48.900724
2020-11-04T20:52:49.268925
2020-11-04T20:52:49.988186
2020-11-04T20:52:50.156459
2020-11-04T20:52:50.431416
2020-11-04T20:52:50.841372
2020-11-04T20:52:51.420453
2020-11-04T20:52:51.450349
2020-11-04T20:52:51.774920
2020-11-04T20:52:52.312627
2020-11-04T20:52:52.404748
2020-11-04T20:52:52.988739
2020-11-04T20:52:53.203399
2020-11-04T20:52:53.717243
2020-11-04T20:52:53.763160
2020-11-04T20:52:54.099050
2020-11-04T20:52:54.462654
2020-11-04T20:52:54.814862
2020-11-04T20:52:55.228237
2020-11-04T20:52:55.780934
2020-11-04T20:52:56.149025
2020-11-04T20:52:56.408660
2020-11-04T20:52:56.797619
2020-11-

AttributeError: 'NoneType' object has no attribute 'to_dict'

[]
{'id': 0, 'battery_function': 0, 'type': 0, 'temperature': 32767, 'current_battery': 6, 'current_consumed': 45, 'energy_consumed': 39, 'battery_remaining': 99, 'voltages[0]': 24196, 'voltages[1]': 65535, 'voltages[2]': 65535, 'voltages[3]': 65535, 'voltages[4]': 65535, 'voltages[5]': 65535, 'voltages[6]': 65535, 'voltages[7]': 65535, 'voltages[8]': 65535, 'voltages[9]': 65535}


In [144]:
while True:
    msg = mlog.recv_msg().to_dict()
    mavpackettype = msg["mavpackettype"]

    #if not (msg["mavpackettype"] == "PARAM_VALUE"):

    #if (mavpackettype == "VFR_HUD"):
    #if (mavpackettype == "WIND"):
    if (mavpackettype == "BATTERY_STATUS"):
        msg.pop('mavpackettype', None)
        break